# LIBRARIES

In [ ]:
import os
import ast
import math
import ccxt
import time
import pickle
import numpy as np
import configparser
import pandas as pd
import google.cloud
from pandas_gbq import gbq
import matplotlib.cm as cm
from boruta import BorutaPy
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
from google.cloud import bigquery
from sklearn import preprocessing
from IPython.display import clear_output
from sklearn.metrics import confusion_matrix
from scipy.interpolate import UnivariateSpline
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

#_____

pd.set_option('display.max_rows', 5000)
pd.options.mode.chained_assignment = None

# FUNCTIONS

In [ ]:
#FUNCIÓN QUE LEE EL ARCHIVO CONFIG
def get_config(category, key):
    
    global config
    return config[category][key]

#_____

#FUNCIÓN QUE GENERA CONEXIÓN CON ARCHIVO CONFIG
def updateConfig(config_name):
    
    global config
    
    config = configparser.ConfigParser()
    config.sections()
    config.read(config_name)
    
#_____

#FUNCIÓN QUE GRAFICA POR COLORES SEGUN COMPRAS Y NO COMRPAS
def plot_colourline(x,y,c):
    c = c
    ax = plt.gca()
    for i in np.arange(len(x)-1):
        ax.plot([x[i],x[i+1]], [y[i],y[i+1]], c=c[i])
    return

#_____

#FUNCIÓN QUE ESCRIBE PICKLE DE VARIABLE
def writePickleVariable(variable,variable_name):
    pickle_out = open(variable_name+".pickle","wb")
    pickle.dump(variable, pickle_out)
    pickle_out.close()
    
#_____
    
#FUNCIÓN QUE LEE PICKLE DE VARIABLE
def readPickleVariable(variable_name):    
    return pickle.load(open(variable_name+".pickle","rb"))

#_____

#FUNCIÓN QUE REDONDEA FLOAT HACIA ABAJO SEGÚN DECIMALES
def round_decimals_down(number:float, decimals:int=2):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.floor(number)

    factor = 10 ** decimals
    return math.floor(number * factor) / factor

#_____

#FUNCIÓN QUE REDONDEA FLOAT HACIA ARRIBA SEGÚN DECIMALES
def round_decimals_up(number:float, decimals:int=2):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.ceil(number)

    factor = 10 ** decimals
    return math.ceil(number * factor) / factor

#_____

#FUNCIÓN QUE DESCARGA BASE DE DATOS DE MERCADO DE BIGQUERY
def downloadDataBaseBigQuery(market_model,time,rows=None):
    
    #_____GOOGLE CLOUD CONECTION
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ubuntu/SurfNet/predictionModels/PREDICTION/bigQueryAccess.json"

    #_____SI NO SE DA UN NÚMERO ESPECÍFICO DE FILAS POR PARÁMETRO
    if rows==None:

        #_____DESCARGAR BASE DE DATOS
        candlesDataBase_BigQuery=gbq.read_gbq("SELECT * FROM [dogwood-terra-308100:surfNet."+market_model.replace("/","_")+"] ORDER BY TIME DESC",project_id="dogwood-terra-308100",dialect="legacy").sort_values(by="TIME")
        candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    else:

        #_____DESCARGAR BASE DE DATOS
        candlesDataBase_BigQuery=gbq.read_gbq("SELECT * FROM [dogwood-terra-308100:surfNet."+market_model.replace("/","_")+"] ORDER BY TIME DESC LIMIT "+str(rows),project_id="dogwood-terra-308100",dialect="legacy").sort_values(by="TIME")
        candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    candlesDataBase_BigQuery.OPEN=candlesDataBase_BigQuery.OPEN.astype(float)
    candlesDataBase_BigQuery.HIGH=candlesDataBase_BigQuery.HIGH.astype(float)
    candlesDataBase_BigQuery.LOW=candlesDataBase_BigQuery.LOW.astype(float)
    candlesDataBase_BigQuery.CLOSE=candlesDataBase_BigQuery.CLOSE.astype(float)
    candlesDataBase_BigQuery.VOLUME=candlesDataBase_BigQuery.VOLUME.astype(float)

    #_____REESTRUCTURAR BASE DE DATOS
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    if time != "1":

        #_____UPDATE CONFIG PREDICTION
        updateConfig("/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini")

        #_____CREAR COLUMNAS
        candlesDataBase_BigQuery_copy=candlesDataBase_BigQuery.copy()
        candlesDataBase_BigQuery_copy=candlesDataBase_BigQuery_copy.set_index('TIME')
        candlesDataBase_BigQuery_copy_total=candlesDataBase_BigQuery_copy[["ID"]].groupby(pd.Grouper(freq=time+'min', offset=time+'min')).min()
        candlesDataBase_BigQuery_copy_total=candlesDataBase_BigQuery_copy_total.join(candlesDataBase_BigQuery_copy[["VOLUME"]].groupby(pd.Grouper(freq=time+'min', offset=time+'min')).sum())
        candlesDataBase_BigQuery_copy_total=candlesDataBase_BigQuery_copy_total.join(candlesDataBase_BigQuery_copy[["LOW"]].groupby(pd.Grouper(freq=time+'min', offset=time+'min')).min())
        candlesDataBase_BigQuery_copy_total=candlesDataBase_BigQuery_copy_total.join(candlesDataBase_BigQuery_copy[["HIGH"]].groupby(pd.Grouper(freq=time+'min', offset=time+'min')).max())

        #_____AGREAR DATOS
        for i in list(candlesDataBase_BigQuery_copy_total.index.values):
            try:
                candlesDataBase_BigQuery_copy_total.at[i,"OPEN"]=candlesDataBase_BigQuery.loc[candlesDataBase_BigQuery.TIME==i]["OPEN"].values[0]
                final_time = i + np.timedelta64(int(time),'m')
                if final_time in list(candlesDataBase_BigQuery_copy_total.index.values):
                    candlesDataBase_BigQuery_copy_total.at[i,"CLOSE"]=candlesDataBase_BigQuery.loc[candlesDataBase_BigQuery.TIME==final_time]["OPEN"].values[0]
                else:
                    candlesDataBase_BigQuery_copy_total.at[i,"CLOSE"]=candlesDataBase_BigQuery.at[len(candlesDataBase_BigQuery)-1,"CLOSE"]
            except:
                pass

        #_____RESET INDEX
        candlesDataBase_BigQuery_copy_total.reset_index(level=['TIME'],inplace=True)

        #_____COLUMNA MARKET
        for i in range(0,len(candlesDataBase_BigQuery_copy_total)):
            candlesDataBase_BigQuery_copy_total.at[i,"MARKET"]=candlesDataBase_BigQuery.MARKET.values[0]

        #_____ORDENAR COLUMNAS
        column_names = list(candlesDataBase_BigQuery.columns)
        candlesDataBase_BigQuery_copy_total = candlesDataBase_BigQuery_copy_total.reindex(columns=column_names)
        candlesDataBase_BigQuery_copy_total.dropna(inplace=True)
        candlesDataBase_BigQuery_copy_total.reset_index(inplace=True,drop=True)
        candlesDataBase_BigQuery=candlesDataBase_BigQuery_copy_total.copy()

    #_____RETURN
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE DEVUELVE EL RSI DADO UN PERIODO DETERMINADO
def RSI(candlesDataBase_BigQuery):
    
    global config
    updateConfig("/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini")
    
    series=pd.Series(list(candlesDataBase_BigQuery.CLOSE.values))
    period=int(get_config("PARAMETERS","RSI_LEN"))
    
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = pd.DataFrame.ewm(u, com=period-1, adjust=False).mean() / \
         pd.DataFrame.ewm(d, com=period-1, adjust=False).mean()
    
    rsi=100 - 100 / (1 + rs)
    
    for i in list(rsi.index.values):
        candlesDataBase_BigQuery.at[i,"RSI"]=rsi[i]
                                        
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE DEVUELVE EL RSI DADO UN PERIODO DETERMINADO
def RSI_GENERAL(series, period):
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = pd.DataFrame.ewm(u, com=period-1, adjust=False).mean() / \
         pd.DataFrame.ewm(d, com=period-1, adjust=False).mean()
    return 100 - 100 / (1 + rs)

#_____

#FUNCIÓN QUE CALCULA MACD
def MACD(candlesDataBase_BigQuery):
    
    global config
    updateConfig("/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini")
    
    close_data_df=candlesDataBase_BigQuery.copy()
    close_data_df.CLOSE=close_data_df.CLOSE.astype(float)
    close_data_df=close_data_df[["CLOSE"]]
    
    #_____CREAR VARIABLE MACD
    macd_12 = close_data_df.ewm(span=int(get_config("PARAMETERS","MACD_SHORT")), adjust=False).mean()
    macd_26 = close_data_df.ewm(span=int(get_config("PARAMETERS","MACD_LONG")), adjust=False).mean()
    macd = macd_12 - macd_26
    macd_9 = macd.ewm(span=int(get_config("PARAMETERS","MACD_TRIGGER")), adjust=False).mean()

    #_____AGREGAR VARIABLES A LA BASE DE DATOS
    candlesDataBase_BigQuery["MACD"]=macd
    candlesDataBase_BigQuery["MACD_"+get_config("PARAMETERS","MACD_SHORT")]=macd_12
    candlesDataBase_BigQuery["MACD_"+get_config("PARAMETERS","MACD_LONG")]=macd_26
    candlesDataBase_BigQuery["MACD_TRIGGER"]=macd_9
    
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE BRINDA LOS NIVELES DE FIBONACCI
def fibonacciLevels(candlesDataBase_BigQuery):
    
    global config
    global fractal
    
    #_____UPDATE CONFIG PREDICTION
    updateConfig("/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini")
    
    #_____ACTUALIZAR PARÁMETROS
    FIBONACCI_PERIOD=int(int(get_config("PARAMETERS","FIBONACCI_PERIOD"))/int(fractal))

    #_____RECORRER DATOS DE TEST
    for i in range(FIBONACCI_PERIOD,len(candlesDataBase_BigQuery)):
        
        #_____FIBONACCI
        lows=list(candlesDataBase_BigQuery[i-FIBONACCI_PERIOD:i].LOW.values)
        highs=list(candlesDataBase_BigQuery[i-FIBONACCI_PERIOD:i].HIGH.values)
        Price_Min = min(lows)
        Price_Max = max(highs)
        Diff = Price_Max-Price_Min
        level1 = Price_Max - 0.236 * Diff
        level2 = Price_Max - 0.382 * Diff
        level3 = Price_Max - 0.618 * Diff
        level4 = Price_Max - 0.786 * Diff
        
        candlesDataBase_BigQuery.at[i,"FIBO_LEVEL_1"]=level1
        candlesDataBase_BigQuery.at[i,"FIBO_LEVEL_2"]=level2
        candlesDataBase_BigQuery.at[i,"FIBO_LEVEL_3"]=level3
        candlesDataBase_BigQuery.at[i,"FIBO_LEVEL_4"]=level4
            
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE CREA VARIABLES ENCONTRADAS EN ARTÍCULOS
def newVariables(candlesDataBase_BigQuery):
    
    #_____VARIABLES NUEVAS
    candlesDataBase_BigQuery['SMA5_VOL'] = candlesDataBase_BigQuery.groupby('MARKET')['VOLUME'].transform(lambda x: x.rolling(window = 5).mean())
    candlesDataBase_BigQuery['SMA15_VOL'] = candlesDataBase_BigQuery.groupby('MARKET')['VOLUME'].transform(lambda x: x.rolling(window = 15).mean())
    candlesDataBase_BigQuery['SMA_VOL_RATIO'] = candlesDataBase_BigQuery['SMA5_VOL']/candlesDataBase_BigQuery['SMA15_VOL']

    #_____VARIABLES NUEVAS
    candlesDataBase_BigQuery['LOWEST_5D'] = candlesDataBase_BigQuery.groupby('MARKET')['LOW'].transform(lambda x: x.rolling(window = 5).min())
    candlesDataBase_BigQuery['HIGH_5D'] = candlesDataBase_BigQuery.groupby('MARKET')['HIGH'].transform(lambda x: x.rolling(window = 5).max())
    candlesDataBase_BigQuery['LOWEST_15D'] = candlesDataBase_BigQuery.groupby('MARKET')['LOW'].transform(lambda x: x.rolling(window = 15).min())
    candlesDataBase_BigQuery['HIGH_15D'] = candlesDataBase_BigQuery.groupby('MARKET')['HIGH'].transform(lambda x: x.rolling(window = 15).max())
    candlesDataBase_BigQuery['STOCHASTIC_5'] = ((candlesDataBase_BigQuery['CLOSE'] - candlesDataBase_BigQuery['LOWEST_5D'])/(candlesDataBase_BigQuery['HIGH_5D'] - candlesDataBase_BigQuery['LOWEST_5D']))*100
    candlesDataBase_BigQuery['STOCHASTIC_15'] = ((candlesDataBase_BigQuery['CLOSE'] - candlesDataBase_BigQuery['LOWEST_15D'])/(candlesDataBase_BigQuery['HIGH_15D'] - candlesDataBase_BigQuery['LOWEST_15D']))*100
    candlesDataBase_BigQuery['STOCHASTIC_D_5'] = candlesDataBase_BigQuery['STOCHASTIC_5'].rolling(window = 5).mean()
    candlesDataBase_BigQuery['STOCHASTIC_D_15'] = candlesDataBase_BigQuery['STOCHASTIC_5'].rolling(window = 15).mean()
    candlesDataBase_BigQuery['STOCHASTIC_RATIO'] = candlesDataBase_BigQuery['STOCHASTIC_D_5']/candlesDataBase_BigQuery['STOCHASTIC_D_15']
    
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN SUPERTREND
def superTrend(df, atr_period, multiplier):
    
    high = df['HIGH']
    low = df['LOW']
    close = df['CLOSE']
    
    # calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # df['atr'] = df['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    #return pd.DataFrame({
    #    'Supertrend': supertrend,
    #    'Final Lowerband': final_lowerband,
    #    'Final Upperband': final_upperband
    #}, index=df.index)

    df["SUPERTREND"]=supertrend

    return df

#_____

def suaviPlusDerivates(candlesDataBase_BigQuery):
    
    global config

    #_____UPDATE CONFIG PREDICTION
    updateConfig("/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini")

    short_ma_hyper=int(get_config("PARAMETERS","SHORT_MA_HYPER"))
    short_ma_normal=int(get_config("PARAMETERS","SHORT_MA_NORMAL"))
    long_ma_normal=int(get_config("PARAMETERS","LONG_MA_NORMAL"))
    long_ma_hyper=int(get_config("PARAMETERS","LONG_MA_HYPER"))
    ultra_long_ma_normal=int(get_config("PARAMETERS","ULTRA_LONG_MA_NORMAL"))

    #_____VECTOR DE PRECIOS
    close_data_df=candlesDataBase_BigQuery.copy()
    close_data_df.CLOSE=close_data_df.CLOSE.astype(float)
    close_data_df=close_data_df[["CLOSE"]]

    #_____SUAVIZACIONES MA & EMA
    close_rolling_short_ma_hyper=close_data_df.rolling(short_ma_hyper).mean()
    close_rolling_short_ma_normal=close_data_df.rolling(short_ma_normal).mean()
    close_rolling_long_ma_normal=close_data_df.ewm(span=long_ma_normal).mean()
    close_rolling_long_ma_hyper=close_data_df.ewm(span=long_ma_hyper).mean()
    close_rolling_ultra_long_ma_normal=close_data_df.ewm(span=ultra_long_ma_normal).mean()

    #_____INSERTAR NUEVAS VARIABLES
    candlesDataBase_BigQuery["ROLLING_SHORT_MA_HYPER"]=close_rolling_short_ma_hyper
    candlesDataBase_BigQuery["ROLLING_SHORT_MA_NORMAL"]=close_rolling_short_ma_normal
    candlesDataBase_BigQuery["ROLLING_LONG_MA_NORMAL"]=close_rolling_long_ma_normal
    candlesDataBase_BigQuery["ROLLING_LONG_MA_HYPER"]=close_rolling_long_ma_hyper
    candlesDataBase_BigQuery["ROLLING_ULTRA_LONG_MA_NORMAL"]=close_rolling_ultra_long_ma_normal

    #_____BOLLINGER
    candlesDataBase_BigQuery['15MA'] = candlesDataBase_BigQuery.groupby('MARKET')['CLOSE'].transform(lambda x: x.rolling(window=15).mean())
    candlesDataBase_BigQuery['SD'] = candlesDataBase_BigQuery.groupby('MARKET')['CLOSE'].transform(lambda x: x.rolling(window=15).std())
    candlesDataBase_BigQuery['UPPERBAND'] = candlesDataBase_BigQuery['15MA'] + 2*candlesDataBase_BigQuery['SD']
    candlesDataBase_BigQuery['LOWERBAND'] = candlesDataBase_BigQuery['15MA'] - 2*candlesDataBase_BigQuery['SD']

    #_____RATE CHANGE
    candlesDataBase_BigQuery['RC'] = candlesDataBase_BigQuery.groupby('MARKET')['CLOSE'].transform(lambda x: x.pct_change(periods = 15))

    #_____ELIMINAR DATOS FALTANTES
    candlesDataBase_BigQuery.dropna(inplace=True)
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    #####

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_CLOSE= np.arange(len(candlesDataBase_BigQuery))
    y_spl_CLOSE = UnivariateSpline(x_CLOSE,candlesDataBase_BigQuery[["CLOSE"]],s=0,k=3)
    Y_1D_CLOSE=y_spl_CLOSE.derivative(n=1)
    Y_1D_CLOSE=pd.DataFrame(Y_1D_CLOSE(x_CLOSE))
    candlesDataBase_BigQuery["Y_1D_CLOSE"]=Y_1D_CLOSE

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_SHORT_MA_HYPER= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_SHORT_MA_HYPER = UnivariateSpline(x_ROLLING_SHORT_MA_HYPER,candlesDataBase_BigQuery[["ROLLING_SHORT_MA_HYPER"]],s=0,k=3)
    Y_1D_ROLLING_SHORT_MA_HYPER=y_spl_ROLLING_SHORT_MA_HYPER.derivative(n=1)
    Y_1D_ROLLING_SHORT_MA_HYPER=pd.DataFrame(Y_1D_ROLLING_SHORT_MA_HYPER(x_ROLLING_SHORT_MA_HYPER))
    candlesDataBase_BigQuery["Y_1D_ROLLING_SHORT_MA_HYPER"]=Y_1D_ROLLING_SHORT_MA_HYPER

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_SHORT_MA_NORMAL= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_SHORT_MA_NORMAL = UnivariateSpline(x_ROLLING_SHORT_MA_NORMAL,candlesDataBase_BigQuery[["ROLLING_SHORT_MA_NORMAL"]],s=0,k=3)
    Y_1D_ROLLING_SHORT_MA_NORMAL=y_spl_ROLLING_SHORT_MA_NORMAL.derivative(n=1)
    Y_1D_ROLLING_SHORT_MA_NORMAL=pd.DataFrame(Y_1D_ROLLING_SHORT_MA_NORMAL(x_ROLLING_SHORT_MA_NORMAL))
    candlesDataBase_BigQuery["Y_1D_ROLLING_SHORT_MA_NORMAL"]=Y_1D_ROLLING_SHORT_MA_NORMAL

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_LONG_MA_NORMAL= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_LONG_MA_NORMAL = UnivariateSpline(x_ROLLING_LONG_MA_NORMAL,candlesDataBase_BigQuery[["ROLLING_LONG_MA_NORMAL"]],s=0,k=3)
    Y_1D_ROLLING_LONG_MA_NORMAL=y_spl_ROLLING_LONG_MA_NORMAL.derivative(n=1)
    Y_1D_ROLLING_LONG_MA_NORMAL=pd.DataFrame(Y_1D_ROLLING_LONG_MA_NORMAL(x_ROLLING_LONG_MA_NORMAL))
    candlesDataBase_BigQuery["Y_1D_ROLLING_LONG_MA_NORMAL"]=Y_1D_ROLLING_LONG_MA_NORMAL

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_LONG_MA_HYPER= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_LONG_MA_HYPER = UnivariateSpline(x_ROLLING_LONG_MA_HYPER,candlesDataBase_BigQuery[["ROLLING_LONG_MA_HYPER"]],s=0,k=3)
    Y_1D_ROLLING_LONG_MA_HYPER=y_spl_ROLLING_LONG_MA_HYPER.derivative(n=1)
    Y_1D_ROLLING_LONG_MA_HYPER=pd.DataFrame(Y_1D_ROLLING_LONG_MA_HYPER(x_ROLLING_LONG_MA_HYPER))
    candlesDataBase_BigQuery["Y_1D_ROLLING_LONG_MA_HYPER"]=Y_1D_ROLLING_LONG_MA_HYPER

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_ULTRA_LONG_MA_NORMAL= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_ULTRA_LONG_MA_NORMAL = UnivariateSpline(x_ROLLING_ULTRA_LONG_MA_NORMAL,candlesDataBase_BigQuery[["ROLLING_ULTRA_LONG_MA_NORMAL"]],s=0,k=3)
    Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL=y_spl_ROLLING_ULTRA_LONG_MA_NORMAL.derivative(n=1)
    Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL=pd.DataFrame(Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL(x_ROLLING_ULTRA_LONG_MA_NORMAL))
    candlesDataBase_BigQuery["Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL"]=Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL

    #####

    #_____CALCULAR VARIABLE RSI
    RSI_ROLLING_SHORT_MA_HYPER = RSI_GENERAL(pd.Series(list(candlesDataBase_BigQuery.ROLLING_SHORT_MA_HYPER.values)), int(get_config("PARAMETERS","RSI_LEN")))
    RSI_ROLLING_SHORT_MA_NORMAL = RSI_GENERAL(pd.Series(list(candlesDataBase_BigQuery.ROLLING_SHORT_MA_NORMAL.values)), int(get_config("PARAMETERS","RSI_LEN")))
    RSI_ROLLING_LONG_MA_NORMAL = RSI_GENERAL(pd.Series(list(candlesDataBase_BigQuery.ROLLING_LONG_MA_NORMAL.values)), int(get_config("PARAMETERS","RSI_LEN")))
    RSI_ROLLING_LONG_MA_HYPER = RSI_GENERAL(pd.Series(list(candlesDataBase_BigQuery.ROLLING_LONG_MA_HYPER.values)), int(get_config("PARAMETERS","RSI_LEN")))
    RSI_ROLLING_ULTRA_LONG_MA_NORMAL = RSI_GENERAL(pd.Series(list(candlesDataBase_BigQuery.ROLLING_ULTRA_LONG_MA_NORMAL.values)), int(get_config("PARAMETERS","RSI_LEN")))

    #_____AGRAGAR VARIABLE RSI A BASE DE DATOS
    candlesDataBase_BigQuery["RSI_ROLLING_SHORT_MA_HYPER"]=RSI_ROLLING_SHORT_MA_HYPER
    candlesDataBase_BigQuery["RSI_ROLLING_SHORT_MA_NORMAL"]=RSI_ROLLING_SHORT_MA_NORMAL
    candlesDataBase_BigQuery["RSI_ROLLING_LONG_MA_NORMAL"]=RSI_ROLLING_LONG_MA_NORMAL
    candlesDataBase_BigQuery["RSI_ROLLING_LONG_MA_HYPER"]=RSI_ROLLING_LONG_MA_HYPER
    candlesDataBase_BigQuery["RSI_ROLLING_ULTRA_LONG_MA_NORMAL"]=RSI_ROLLING_ULTRA_LONG_MA_NORMAL

    #_____REESTRUCTURAR BASE DE DATOS
    candlesDataBase_BigQuery=candlesDataBase_BigQuery[int(get_config("PARAMETERS","ULTRA_LONG_MA_NORMAL"))+1:]
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
    
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE CALCULA MACD
def LAGS(candlesDataBase_BigQuery):
    
    for j in list(candlesDataBase_BigQuery.columns)[3:]:
        candlesDataBase_BigQuery[j+"_1"]=candlesDataBase_BigQuery[j].shift(1)
        candlesDataBase_BigQuery[j+"_2"]=candlesDataBase_BigQuery[j].shift(2)
        candlesDataBase_BigQuery[j+"_3"]=candlesDataBase_BigQuery[j].shift(3)
        
    candlesDataBase_BigQuery.dropna(inplace=True)
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
        
    return candlesDataBase_BigQuery

#_____

#CREATE VARIABLES
def variableCreation(candlesDataBase_BigQuery):
    
    global fractal
    updateConfig("/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini")
    
    candlesDataBase_BigQuery=RSI(candlesDataBase_BigQuery)
    candlesDataBase_BigQuery=MACD(candlesDataBase_BigQuery)
#     candlesDataBase_BigQuery=fibonacciLevels(candlesDataBase_BigQuery)
#     candlesDataBase_BigQuery=newVariables(candlesDataBase_BigQuery)
    candlesDataBase_BigQuery=superTrend(candlesDataBase_BigQuery, int(get_config("PARAMETERS","ATR_PERIOD")), int(get_config("PARAMETERS","ATR_MULTIP")))
#     candlesDataBase_BigQuery=suaviPlusDerivates(candlesDataBase_BigQuery)
#     candlesDataBase_BigQuery=LAGS(candlesDataBase_BigQuery)
    
    candlesDataBase_BigQuery.dropna(inplace=True)
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
    
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE SIMULA COMPRA/VENTA MARKET EN BINANCE SEGÚN MERCADO -> SI SIDE==BUY, TRADINGAMOUNT==USDT // SI SIDE==SELL, TRADINGAMOUNT==CRYPTO
def simulateMarket(exchange,market,tradingAmount,side,comission):
    
    while True:
        
        try:
    
            #EN CASO DE QUE SEA COMPRA
            if side=="buy":

                #VALIDAR QUE ORDERBOOK ALCANCE PARA HACER LA SIMULACIÓN
                while_break=0
                while while_break<tradingAmount*1.5:
                    #DESCARGAR BUYINGBOOK
                    buyingBook=pd.DataFrame(exchange.fetch_order_book(market)["asks"],columns=["ASK_VALUE","ASK_SIZE"])
                    buyingBook["ASK_PRICE"]=buyingBook["ASK_VALUE"]*buyingBook["ASK_SIZE"]

                    #ACTUALIZAR SUMA DE VOLUMENES DE ORDENES EN ORDERBOOK DESCARGADO
                    while_break=sum(buyingBook.ASK_PRICE.values)

                    #SI VOLUMEN TOTAL ES MAYOR A LO BUSCADO, SEGUIR CON SIMULACIÓN -> DLC, PRINT + ESPERAR
                    if while_break<tradingAmount*1.5:
                        print("[ERROR]: buyingBook(binance,"+market+","+str(tradingAmount)+")")
                        time.sleep(1)

                #_____

                #SIMULACIÓN DE COMPRA CON USDT -> RETORNA VALOR DE CRYPTO ADQUIRIDO NETO
                usdt_used=0
                crypto_buyed=0
                usdt_used_acum=0
                crypto_buyed_acum=0
                contador_buying_book=0

                while usdt_used_acum<tradingAmount:

                    usdt_used=buyingBook.at[contador_buying_book,"ASK_PRICE"]
                    crypto_buyed=buyingBook.at[contador_buying_book,"ASK_SIZE"]
                        
                    #_____SIMULACIÓN
                    if usdt_used_acum+usdt_used < tradingAmount:
                        usdt_used_acum=usdt_used_acum+usdt_used
                        crypto_buyed_acum=crypto_buyed_acum+crypto_buyed
                    else:
                        diferencia=tradingAmount-usdt_used_acum
                        usdt_used_acum=usdt_used_acum+diferencia

                        porcentaje_diferencia=diferencia/usdt_used
                        crypto_buyed_acum=crypto_buyed_acum+crypto_buyed*porcentaje_diferencia
                        
                
                #_____REVISAR DECIMALES
                decimales=0
                for i in range(0,10):
                    crypto_buyed=buyingBook.at[i,"ASK_SIZE"]
                    if marketFloats(crypto_buyed)>decimales:
                        decimales=marketFloats(crypto_buyed)

                #crypto_buyed_acum=round_decimals_down(crypto_buyed_acum*(1-comission),decimales)
                crypto_buyed_acum=round_decimals_down(crypto_buyed_acum,decimales)
                
                response = crypto_buyed_acum

            #_____

            #EN CASO DE QUE SEA VENTA
            if side=="sell":

                #VALIDAR QUE ORDERBOOK ALCANCE PARA HACER LA SIMULACIÓN
                while_break=0
                while while_break<tradingAmount*1.5:
                    #DESCARGAR SELLINGBOOK
                    sellingBook=pd.DataFrame(exchange.fetch_order_book(market)["bids"],columns=["BID_VALUE","BID_SIZE"])

                    #ACTUALIZAR SUMA DE VOLUMENES DE ORDENES EN ORDERBOOK DESCARGADO
                    while_break=sum(sellingBook.BID_SIZE.values)

                    #SI VOLUMEN TOTAL ES MAYOR A LO BUSCADO, SEGUIR CON SIMULACIÓN -> DLC, PRINT + ESPERAR
                    if while_break<tradingAmount*1.5:
                        print("[ERROR]: buyingBook("+exchange.name+","+market+","+str(tradingAmount)+")")
                        time.sleep(1)

                #_____

                #INICIALIZAR PARÁMETROS PARA LA SIMULACIÓN
                sellAmount=0.0
                sellCounter=0
                sellPriceACUM=0.0
                sellAmountACUM=0.0

                #SIMULAR VENTA
                while sellAmountACUM<tradingAmount:
                    sellAmount=float(sellingBook.at[sellCounter,"BID_SIZE"])
                    sellAmountACUM=sellAmountACUM+sellAmount
                    if sellAmountACUM<tradingAmount:
                        sellPriceACUM=sellPriceACUM+(sellAmount*float(sellingBook.iloc[sellCounter,0]))
                        sellCounter=sellCounter+1
                    else:
                        printCRY=sellAmountACUM-sellAmount+tradingAmount-(sellAmountACUM-sellAmount)
                        sellPriceACUM=sellPriceACUM+((tradingAmount-(sellAmountACUM-sellAmount))*float(sellingBook.iloc[sellCounter,0]))

                response = sellPriceACUM*(1-comission)
            
            break
            
        except Exception as e: 
            print(f"[ERROR]: simulateMarket() {e}")
            time.sleep(1)
            
    return response
            
#_____

#FUNCIÓN QUE CREA UNA COMPRA TIPO MARKET
def createMarketBuy(market,money):
    
    global amount
    global binance
    global comission
    global marketBuyOrder
    
    amount=simulateMarket(binance,market,money,"buy",comission)
    marketBuyOrder=binance.create_market_buy_order(symbol=market,amount=amount)
            
#_____

#FUNCIÓN QUE CREA ORDEN DE COMPRA MARKET Y ACTUALIZA BASE DE DATOS DE TRADEO
def buyMyFriend(market,money):
    
    global decimales
    global comission
    global row_index
    global cryptoAmount
    global marketBuyCost
    global marketBuyOrder
    global marketSellPrice
    global marketBuyOrder_ID
    global tradingBook_MonoBot
    
    #_____LEER BASE DE DATOS
    tradingBook_MonoBot=readPickleVariable("tradingBook_MonoBot")

    #_____DETERMINAR FILA DE INTEGRACIÓN DE DATOS
    row_index=len(tradingBook_MonoBot)

    #_____CREAR COMPRA MARKET
    createMarketBuy(market,money)

    #_____AGREGAR DATOS A BASE DE DATOS DE TRADEO
    tradingBook_MonoBot.at[row_index,"MACHINE"]="manual"
    tradingBook_MonoBot.at[row_index,"TRANSACTION_ID"]=marketBuyOrder["info"]["transactTime"]
    tradingBook_MonoBot.at[row_index,"MARKET"]=market
    tradingBook_MonoBot.at[row_index,"BUY_ID"]=marketBuyOrder["id"]
    tradingBook_MonoBot.at[row_index,"DATE_BUY"]=str(datetime.fromtimestamp(int(marketBuyOrder["info"]["transactTime"])/1000))
    tradingBook_MonoBot.at[row_index,"BUY_CRYPTO_AMOUNT"]=float(marketBuyOrder["info"]["executedQty"])
    
    try:
        tradingBook_MonoBot.at[row_index,"BUY_FEE"]=float(marketBuyOrder["fee"]["cost"])
        tradingBook_MonoBot.at[row_index,"BUY_CURRENCY_FEE"]=marketBuyOrder["fee"]["currency"]
    except:
        tradingBook_MonoBot.at[row_index,"BUY_FEE"]=comission*tradingBook_MonoBot.at[row_index,"BUY_CRYPTO_AMOUNT"]
        tradingBook_MonoBot.at[row_index,"BUY_CURRENCY_FEE"]=market
    
    
    #_____GUARDAR ID PARA FUTURO CRUCE CON VENTA
    marketBuyOrder_ID=marketBuyOrder["id"]
    
    #_____ACTUALIZAR PRECIOS DE VENTA
    tradingBook_MonoBot.at[row_index,"BUY_FEE_PERCET"]=comission
    tradingBook_MonoBot.at[row_index,"NET_BUY_COST"]=float(marketBuyOrder["cost"])*(1+comission)
    cryptoAmount=tradingBook_MonoBot.at[row_index,"BUY_CRYPTO_AMOUNT"]
    cryptoAmount=round_decimals_down(cryptoAmount,decimales)
    marketBuyCost=tradingBook_MonoBot.at[row_index,"NET_BUY_COST"]
    
#_____

#FUNCIÓN QUE RETORNA BALANCE USDC EN BINANCE
def getCryptoBinance(market):
    global binance
    
    my_market=market.replace("/USDT","")
    while True:
        try:
            return float(binance.fetch_balance()[my_market]["total"])
            break
        except:
            print("ERROR: getCryptoBinance()")
            time.sleep(1)
            
#_____

#FUNCIÓN QUE CREA ORDEN MARKET DE VENTA
def createMarketSell(market):
    
    global check
    global amount
    global binance
    global decimales
    global cryptoAmount
    global marketSellOrder
    
    check=getCryptoBinance(market)
    amount=round_decimals_down(getCryptoBinance(market),decimales)
    #marketSellOrder=binance.create_market_sell_order(symbol=market,amount=amount)
    #marketSellOrder=binance.create_market_sell_order(symbol=market,amount=cryptoAmount)
    try:
        marketSellOrder=binance.create_market_sell_order(symbol=market,amount=amount)
    except:
        if (getCryptoBinance(market)==0) and (type(marketSellOrder)==dict):
            pass
        else:
            ERROR_createMarketSell
    
#_____

#FUNCIÓN QUE CREA ORDEN DE VENTA MARKET Y ACTUALIZA BASE DE DATOS DE TRADEO
def sellMyFriend(market):
    
    global comission
    global marketSellOrder
    global marketBuyOrder_ID
    global tradingBook_MonoBot

    #_____DETERMINAR FILA DE INTEGRACIÓN DE DATOS
    row_index=tradingBook_MonoBot.loc[tradingBook_MonoBot.BUY_ID==marketBuyOrder_ID].index.values[0]

    #_____CREAR VENTA MARKET
    createMarketSell(market)

    #_____AGREGAR DATOS A BASE DE DATOS DE TRADEO
    tradingBook_MonoBot.at[row_index,"SELL_ID"]=marketSellOrder["id"]
    tradingBook_MonoBot.at[row_index,"DATE_SELL"]=str(datetime.fromtimestamp(int(marketSellOrder["info"]["transactTime"])/1000))
    tradingBook_MonoBot.at[row_index,"SELL_FEE"]=float(marketSellOrder["fee"]["cost"])
    tradingBook_MonoBot.at[row_index,"SELL_CURRENCY_FEE"]=marketSellOrder["fee"]["currency"]
    tradingBook_MonoBot.at[row_index,"SELL_CRYPTO_AMOUNT"]=float(marketSellOrder["info"]["executedQty"])
    
    #_____ACTUALIZAR CONFIG
    updateConfig('config_surfNet_MonoBot.ini')
    
    #_____CALCULAR COMISIÓN
    tradingBook_MonoBot.at[row_index,"NET_SELL_COST"]=float(marketSellOrder["cost"])*(1-comission)
    tradingBook_MonoBot.at[row_index,"NET_UTILITY"]=tradingBook_MonoBot.at[row_index,"NET_SELL_COST"]/tradingBook_MonoBot.at[row_index,"NET_BUY_COST"]-1
    
    #_____ESCRIBIR PICKLE
    writePickleVariable(tradingBook_MonoBot,"tradingBook_MonoBot")
    
#_____

#FUNCIÓN QUE RETORNA BALANCE USDC EN BINANCE
def getUsdtBinance():
    global binance
    while True:
        try:
            return float(binance.fetch_balance()["USDT"]["total"])
            break
        except:
            print("ERROR: getUsdtBinance()")
            time.sleep(1)
            
#_____

#FUNCIÓN QUE ESTIMA LA CANTIDAD DE DECIMALES A REDONDEAR
def marketFloats(value): 
    Completo = str(value)
    Entero= str(math.floor(value))
    lenCompleto = len(Completo)
    lenEntero = len(Entero)
    if lenEntero != lenCompleto:
        decimalVector=Completo.split(".")
        for i in decimalVector[1]:
            if i != "0":
                tengoDecimales=True
            else:
                tengoDecimales=False
        if tengoDecimales:
            decimales=lenCompleto-lenEntero-1
        else:
            decimales=0
    else:
        decimales = 0
    
    return decimales

# PARAMETERS

In [ ]:
#_____ACTUALIZAR ARCHIVO CONFIG
updateConfig('config_surfNet_MonoBot.ini')

#_____BINANCE
binance = ccxt.binance()
binance.apiKey = "AClVXKNa29UOHZLQJOclsqxmIwB3ytGiLdqJv9tRqDAnBa1igFrR8lZdii77rto1"
binance.secret = "VxNazwB9y57Kz4ydF4IFckypW3WYM90qjWzb925itZPU909VKjGrR4MBKvMevYZz"
binance.options['createMarketBuyOrderRequiresPrice'] = False

In [ ]:
#_____BASE DE DATOS LIMPIA
# tradingBook_MonoBot=pd.DataFrame(columns=["MACHINE","TRANSACTION_ID","MARKET","BUY_ID","DATE_BUY","SELL_ID","DATE_SELL","BUY_CRYPTO_AMOUNT","NET_BUY_COST","BUY_FEE","BUY_FEE_PERCET","BUY_CURRENCY_FEE","SELL_CRYPTO_AMOUNT","NET_SELL_COST","SELL_FEE","SELL_CURRENCY_FEE","NET_UTILITY"])
# writePickleVariable(tradingBook_MonoBot,"tradingBook_MonoBot")

# PORTFOLIO

In [ ]:
#_____PORTFOLIO
portfolio=readPickleVariable("/home/ubuntu/SurfNet/surfNetRun/portfolio")
portfolio.reset_index(inplace=True,drop=True)

#_____RELEVANT VARIABLES
contador=0
for market_model in list(portfolio.MARKET.values):

    #_____CARGAR BASE DE DATOS
    myData = binance.fetch_ohlcv(market_model,"15"+"m",limit=1000)
    candlesDataBase_BigQuery =pd.DataFrame(columns=["ID","TIME","MARKET","OPEN","HIGH","LOW","CLOSE","VOLUME"])

    for i in range(0,len(myData)):
        candlesDataBase_BigQuery.at[i,"ID"]=myData[i][0]
        candlesDataBase_BigQuery.at[i,"TIME"]=datetime.fromtimestamp(myData[i][0]/1000.0)
        candlesDataBase_BigQuery.at[i,"MARKET"]=market_model
        candlesDataBase_BigQuery.at[i,"OPEN"]=myData[i][1]
        candlesDataBase_BigQuery.at[i,"HIGH"]=myData[i][2]
        candlesDataBase_BigQuery.at[i,"LOW"]=myData[i][3]
        candlesDataBase_BigQuery.at[i,"CLOSE"]=myData[i][4]
        candlesDataBase_BigQuery.at[i,"VOLUME"]=myData[i][5]

    #_____GUARDAR BASE DE DATOS PARA BACKTESTING FUTURO
    backtestingDataFrame=candlesDataBase_BigQuery.copy()

    #_____CREAR VARIABLES INPUT
    candlesDataBase_BigQuery=variableCreation(candlesDataBase_BigQuery)

    #####
    
    portfolio.at[contador,"RSI"]=candlesDataBase_BigQuery.at[len(candlesDataBase_BigQuery)-1,"RSI"]
    portfolio.at[contador,"MACD"]=candlesDataBase_BigQuery.at[len(candlesDataBase_BigQuery)-1,"MACD"]
    portfolio.at[contador,"MACD_TRIGGER"]=candlesDataBase_BigQuery.at[len(candlesDataBase_BigQuery)-1,"MACD_TRIGGER"]
    portfolio.at[contador,"SUPERTREND"]=candlesDataBase_BigQuery.at[len(candlesDataBase_BigQuery)-1,"SUPERTREND"]
    
    #####
    
    checks=0
    if portfolio.at[contador,"RSI"]<50:
        checks+=1
    if (portfolio.at[contador,"MACD"]<0) and (portfolio.at[contador,"MACD_TRIGGER"]<0) and (portfolio.at[contador,"MACD"]/portfolio.at[contador,"MACD_TRIGGER"]>=0.9) and (portfolio.at[contador,"MACD"]/portfolio.at[contador,"MACD_TRIGGER"]<=1.1):
        checks+=1
    if portfolio.at[contador,"SUPERTREND"]==True:
        checks+=1
    
    portfolio.at[contador,"BUY_FACTOR"]=int(checks)
    
    #####
    
    contador+=1
    
portfolio

# BUY

PARAMETERS

In [ ]:
comission=0.001
market="ETH/USDT"
money=49000

COGITO ERGO SUM

In [ ]:
#_____COMPRAR
buyMyFriend(market,money)
ejecuto_venta=0

#_____NO PARAR HASTA VENDER
while ejecuto_venta==0:
    
    updateConfig("monoManual.ini")

    #_____SLEEP
    time.sleep(5)
    
    #_____ACTUALIZAR PRECIO SIMULADO DE VENTA                                                        
    precio_venta_simulado=simulateMarket(binance,market,cryptoAmount,"sell",comission)
    margenUtilidad=(precio_venta_simulado/marketBuyCost)-1
    
    #_____PRINT
    clear_output()
    print(market,margenUtilidad)

    #_____SI MARGEN DE UTILIDAD CALCULADO LLEGA A ALGUNO DE LOS LÍMITES
    if (margenUtilidad >= float(get_config("PARAMETERS","TP"))) or (margenUtilidad <= float(get_config("PARAMETERS","SL"))):

        #_____EJECUTAR ORDEN DE VENTA
        sellMyFriend(market)

BALANCEAR BNB 

In [ ]:
# #_____DECIMALES BNB
# buyingBook=pd.DataFrame(binance.fetch_order_book("BNB/USDT")["asks"],columns=["ASK_VALUE","ASK_SIZE"])
# decimales=0
# for i in range(0,10):
#     crypto_buyed=buyingBook.at[i,"ASK_SIZE"]
#     if marketFloats(crypto_buyed)>decimales:
#         decimales=marketFloats(crypto_buyed)
# #_____VALIDAR MONTO
# # amount_BNB=round_decimals_down(simulateMarket(binance,"BNB/USDT",getUsdtBinance()*0.002,"buy",comission),decimales)
# amount_BNB=round_decimals_down(simulateMarket(binance,"BNB/USDT",getUsdtBinance()*0.0015,"buy",comission),decimales)
# marketBuyOrder_BNB=binance.create_market_buy_order(symbol="BNB/USDT",amount=amount_BNB)